In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

In [2]:
# Create a spark Spark Session
spark = SparkSession \
    .builder \
    .appme("VideoGameEDA_DF") \
    .getOrCreate()

24/09/09 12:41:20 WARN Utils: Your hostname, riddhi-pc resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/09/09 12:41:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/09 12:41:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Show spark info
spark

In [4]:
# Read the Video Games Dataset
csvDF = spark.read.options(header = True).csv('Datasets/Video_Games.csv')

In [9]:
# Print the Schema of the dataset
csvDF.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year_of_Release: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: string (nullable = true)
 |-- EU_Sales: string (nullable = true)
 |-- JP_Sales: string (nullable = true)
 |-- Other_Sales: string (nullable = true)
 |-- Global_Sales: string (nullable = true)
 |-- Critic_Score: string (nullable = true)
 |-- Critic_Count: string (nullable = true)
 |-- User_Score: string (nullable = true)
 |-- User_Count: string (nullable = true)
 |-- Developer: string (nullable = true)
 |-- Rating: string (nullable = true)



In [16]:
# type conversion for columns
updatedSchemaDF = csvDF.withColumn("Year_of_Release", csvDF.Year_of_Release.cast('int')) \
.withColumn("NA_Sales", csvDF.NA_Sales.cast('float')) \
.withColumn("EU_Sales", csvDF.EU_Sales.cast('float')) \
.withColumn("JP_Sales", csvDF.JP_Sales.cast('float')) \
.withColumn("Other_Sales", csvDF.Other_Sales.cast('float')) \
.withColumn("Global_Sales", csvDF.Global_Sales.cast('float')) \
.withColumn("Critic_Score", csvDF.Critic_Score.cast('float')) \
.withColumn("Critic_Count", csvDF.Critic_Count.cast('int')) \
.withColumn("User_Count", csvDF.User_Count.cast('int')) \
.withColumn("User_Score", csvDF.User_Score.cast('float')) 

In [17]:
updatedSchemaDF.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Platform: string (nullable = true)
 |-- Year_of_Release: integer (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- NA_Sales: float (nullable = true)
 |-- EU_Sales: float (nullable = true)
 |-- JP_Sales: float (nullable = true)
 |-- Other_Sales: float (nullable = true)
 |-- Global_Sales: float (nullable = true)
 |-- Critic_Score: float (nullable = true)
 |-- Critic_Count: integer (nullable = true)
 |-- User_Score: float (nullable = true)
 |-- User_Count: integer (nullable = true)
 |-- Developer: string (nullable = true)
 |-- Rating: string (nullable = true)



In [24]:
# Calculate the ToTal Sales per Game - North America, EU, JP, Global and Other sales.
from pyspark.sql import functions as F

updatedSchemaDF.groupBy("Name") \
    .agg(F.round(F.sum("NA_Sales"), 2).alias("Total_NA_Sales"), 
         F.round(F.sum("EU_Sales"), 2).alias("Total_EU_Sales"), \
         F.round(F.sum("EU_Sales"), 2).alias("Total_EU_Sales"), \
         F.round(F.sum("JP_Sales"), 2).alias("Total_JP_Sales"), \
         F.round(F.sum("Global_Sales"), 2).alias("Total_Global_Sales"), \
         F.round(F.sum("Other_Sales"), 2).alias("Total_Other_Sales")) \
    .show(truncate = False)

+-----------------------------------------+--------------+--------------+--------------+--------------+------------------+-----------------+
|Name                                     |Total_NA_Sales|Total_EU_Sales|Total_EU_Sales|Total_JP_Sales|Total_Global_Sales|Total_Other_Sales|
+-----------------------------------------+--------------+--------------+--------------+--------------+------------------+-----------------+
|The Elder Scrolls V: Skyrim              |9.65          |8.37          |8.37          |0.39          |21.15             |2.73             |
|The Legend of Zelda: Oracle of Ages      |0.92          |0.53          |0.53          |0.41          |1.92              |0.06             |
|Call of Duty Black Ops: Declassified     |0.71          |0.43          |0.43          |0.07          |1.47              |0.26             |
|Joust                                    |1.01          |0.06          |0.06          |0.0           |1.08              |0.01             |
|Legacy of Ka

In [29]:
# Replace null values in the "Critic_Score" and "User_Score" columns with appropriate values

updatedSchemaDF = updatedSchemaDF.na.fill({"Critic_Score" : 0, \
                         "User_Score" : 0, \
                         "NA_Sales" : 0, \
                         "EU_Sales" : 0, \
                         "JP_Sales" : 0, \
                         "Other_Sales" : 0, \
                         "Global_Sales" : 0, \
                         "Critic_Count" : 0, \
                         "User_Count" : 0, \
                         "Developer" : 'N/A', \
                         "Rating" : 'N/A', \
                         "Publisher" : 'N/A', \
                         "Genre" : 'N/A', \
                         "Platform" : 'N/A', \
})

In [30]:
updatedSchemaDF.show(truncate=False)

+--------------------------------------------+--------+---------------+------------+----------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|Name                                        |Platform|Year_of_Release|Genre       |Publisher             |NA_Sales|EU_Sales|JP_Sales|Other_Sales|Global_Sales|Critic_Score|Critic_Count|User_Score|User_Count|Developer          |Rating|
+--------------------------------------------+--------+---------------+------------+----------------------+--------+--------+--------+-----------+------------+------------+------------+----------+----------+-------------------+------+
|Wii Sports                                  |Wii     |2006           |Sports      |Nintendo              |41.36   |28.96   |3.77    |8.45       |82.53       |76.0        |51          |8.0       |322       |Nintendo           |E     |
|Super Mario Bros.                           |NES     |1985 

In [34]:
# Get total number of the unique Gaming Platforms
updatedSchemaDF \
.agg(F.countDistinct("Platform").alias("Total_Unique_Platform")) \
.show(truncate = False)

+---------------------+
|Total_Unique_Platform|
+---------------------+
|31                   |
+---------------------+



In [36]:
#Generate a dataframe - Find total users per gaming platform 
totalUserPerPlatform = updatedSchemaDF.groupBy("Platform") \
.agg(F.sum("User_Count").alias("Total_Users")) \

totalUserPerPlatform.show(100, truncate = False)

+--------+-----------+
|Platform|Total_Users|
+--------+-----------+
|3DO     |0          |
|PC      |473781     |
|PS3     |180567     |
|NES     |0          |
|PS      |11964      |
|DC      |1183       |
|GEN     |0          |
|PS2     |54186      |
|3DS     |16041      |
|PCFX    |0          |
|GG      |0          |
|WiiU    |22381      |
|SNES    |0          |
|GB      |0          |
|SCD     |0          |
|N64     |0          |
|PS4     |133166     |
|PSP     |12808      |
|2600    |0          |
|XOne    |52010      |
|X360    |180793     |
|GBA     |5856       |
|WS      |0          |
|Wii     |29749      |
|GC      |13084      |
|PSV     |13776      |
|XB      |14216      |
|DS      |15764      |
|TG16    |0          |
|NG      |0          |
|SAT     |0          |
+--------+-----------+



In [39]:
# Identify the top-selling games globally and regionally (NA, EU, JP, Globally, Other) - Top 10.
# 1. NA_Sales

updatedSchemaDF.groupBy("Name") \
.agg(F.round(F.sum("NA_Sales"), 2).alias("Top10_NA_Sales")) \
.orderBy(F.desc("Top10_NA_Sales")) \
.show(10, truncate = False)

+------------------------------+--------------+
|Name                          |Top10_NA_Sales|
+------------------------------+--------------+
|Wii Sports                    |41.36         |
|Super Mario Bros.             |32.48         |
|Duck Hunt                     |26.93         |
|Tetris                        |26.17         |
|Grand Theft Auto V            |23.84         |
|Call of Duty: Black Ops       |17.57         |
|Super Mario World             |15.99         |
|Mario Kart Wii                |15.68         |
|Wii Sports Resort             |15.61         |
|Call of Duty: Modern Warfare 3|15.54         |
+------------------------------+--------------+
only showing top 10 rows



In [40]:
# 2. EU_Sales

updatedSchemaDF.groupBy("Name") \
.agg(F.round(F.sum("EU_Sales"), 2).alias("Top10_EU_Sales")) \
.orderBy(F.desc("Top10_EU_Sales")) \
.show(10, truncate = False)

+------------------------------+--------------+
|Name                          |Top10_EU_Sales|
+------------------------------+--------------+
|Wii Sports                    |28.96         |
|Grand Theft Auto V            |23.42         |
|Mario Kart Wii                |12.76         |
|FIFA 15                       |12.02         |
|Call of Duty: Modern Warfare 3|11.15         |
|FIFA 16                       |11.09         |
|FIFA 14                       |10.96         |
|Nintendogs                    |10.95         |
|Wii Sports Resort             |10.93         |
|Call of Duty: Black Ops II    |10.84         |
+------------------------------+--------------+
only showing top 10 rows



In [41]:
# 3. JP_Sales

updatedSchemaDF.groupBy("Name") \
.agg(F.round(F.sum("JP_Sales"), 2).alias("Top10_JP_Sales")) \
.orderBy(F.desc("Top10_JP_Sales")) \
.show(10, truncate = False)

+----------------------------------+--------------+
|Name                              |Top10_JP_Sales|
+----------------------------------+--------------+
|Pokemon Red/Pokemon Blue          |10.22         |
|Pokemon Gold/Pokemon Silver       |7.2           |
|Super Mario Bros.                 |6.96          |
|New Super Mario Bros.             |6.5           |
|Pokemon Diamond/Pokemon Pearl     |6.04          |
|Tetris                            |6.03          |
|Pokemon Black/Pokemon White       |5.65          |
|Dragon Quest VII: Warriors of Eden|5.4           |
|Pokemon Ruby/Pokemon Sapphire     |5.38          |
|Animal Crossing: Wild World       |5.33          |
+----------------------------------+--------------+
only showing top 10 rows



In [42]:
# 4. Global_Sales

updatedSchemaDF.groupBy("Name") \
.agg(F.round(F.sum("Global_Sales"), 2).alias("Top10_Global_Sales")) \
.orderBy(F.desc("Top10_Global_Sales")) \
.show(10, truncate = False)

+------------------------------+------------------+
|Name                          |Top10_Global_Sales|
+------------------------------+------------------+
|Wii Sports                    |82.53             |
|Grand Theft Auto V            |56.57             |
|Super Mario Bros.             |45.31             |
|Tetris                        |35.84             |
|Mario Kart Wii                |35.52             |
|Wii Sports Resort             |32.77             |
|Pokemon Red/Pokemon Blue      |31.37             |
|Call of Duty: Black Ops       |30.82             |
|Call of Duty: Modern Warfare 3|30.59             |
|New Super Mario Bros.         |29.8              |
+------------------------------+------------------+
only showing top 10 rows



In [43]:
# 5. Other_Sales

updatedSchemaDF.groupBy("Name") \
.agg(F.round(F.sum("Other_Sales"), 2).alias("Top10_Other_Sales")) \
.orderBy(F.desc("Top10_Other_Sales")) \
.show(10, truncate = False)

+------------------------------+-----------------+
|Name                          |Top10_Other_Sales|
+------------------------------+-----------------+
|Grand Theft Auto: San Andreas |10.71            |
|Wii Sports                    |8.45             |
|Grand Theft Auto V            |7.9              |
|Gran Turismo 4                |7.53             |
|Call of Duty: Black Ops II    |3.76             |
|FIFA Soccer 08                |3.52             |
|Pro Evolution Soccer 2008     |3.51             |
|Call of Duty: Black Ops 3     |3.49             |
|Call of Duty: Black Ops       |3.31             |
|Call of Duty: Modern Warfare 3|3.29             |
+------------------------------+-----------------+
only showing top 10 rows



In [61]:
# Popular Genres:Identify which genres are the most popular based on sales.

cols_to_sum = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Global_Sales', 'Other_Sales']
popularGenreDF = updatedSchemaDF \
                    .withColumn("Total_Sales", F.expr('+'.join(cols_to_sum)))

In [70]:
popularGenreDF.select("Genre", "Total_Sales") \
.orderBy(F.desc("Total_Sales")) \
.show(100, truncate = False)

+------------+-----------+
|Genre       |Total_Sales|
+------------+-----------+
|Sports      |165.06999  |
|Platform    |80.48      |
|Racing      |71.04      |
|Sports      |65.54      |
|Role-Playing|62.75      |
|Puzzle      |60.520004  |
|Platform    |59.600002  |
|Misc        |57.829998  |
|Platform    |56.64      |
|Shooter     |56.620003  |
|Simulation  |49.34      |
|Racing      |46.420002  |
|Role-Playing|46.190002  |
|Sports      |45.4       |
|Misc        |43.629997  |
|Sports      |43.59      |
|Action      |42.09      |
|Action      |41.62      |
|Platform    |41.23      |
|Misc        |40.29      |
|Role-Playing|36.49      |
|Platform    |36.28      |
|Platform    |34.559998  |
|Action      |32.54      |
|Action      |32.3       |
|Role-Playing|31.69      |
|Puzzle      |30.57      |
|Role-Playing|30.27      |
|Racing      |29.95      |
|Shooter     |29.46      |
|Role-Playing|29.28      |
|Shooter     |29.260002  |
|Shooter     |29.229998  |
|Role-Playing|29.2       |
|